In [50]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

In [51]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['COHERE_API_KEY'] = userdata.get("COHERE_API_KEY")

In [52]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Step-1: Document Loading

In [53]:
!pip install pypdf --quiet

In [54]:
from langchain.document_loaders import PyPDFLoader

In [55]:
# Fetch the document to analysis the text present
!wget https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/SampleMR.pdf
loader = PyPDFLoader("SampleMR.pdf")
pages = loader.load()
print(len(pages))

--2025-01-03 06:39:01--  https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/SampleMR.pdf
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/main/SampleMR.pdf [following]
--2025-01-03 06:39:01--  https://raw.githubusercontent.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/main/SampleMR.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41628 (41K) [application/octet-stream]
Saving to: ‘SampleMR.pdf.1’

SampleMR.pdf.1      100%[===================>]  40.65K  --.-KB/s    

In [56]:
# Understand number of characters, text, lines, and words present in the document.
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 4
Lines 151
Words 1690
Charecters 9429


# Step-2:Split the data into Chunks

In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [58]:
# Fetch the words and convert them into chunks which can be processed further
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

58


In [59]:
print(chunks[0])

page_content='Full name of patient: Mr Tan Ah Kow 
NRIC/FIN/Passport no. of patient: S1111111X 
Age of patient: 55 years old 
SECTION 2: DOCTOR’S PARTICULARS 
Full name of doctor: Tan Ah Moi' metadata={'source': 'SampleMR.pdf', 'page': 0}


# Step-3: Creating embeddings

In [60]:
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings

In [61]:

embeddings = CohereEmbeddings(user_agent="langchain")

In [62]:
# A sample embedding using Cohere embedding to convert text into numbers
sample_embedding = embeddings.embed_query("Patient is XYZ")
print(sample_embedding)

[0.6166992, 0.21386719, -0.022537231, -1.1689453, -1.0615234, 0.34960938, -1.8095703, -1.0517578, 0.47460938, 1.4521484, 0.7207031, 1.2783203, -1.8427734, -0.5415039, -1.15625, -0.8847656, 0.42773438, 1.4306641, 1.8125, -1.0712891, -1.2890625, -0.41918945, -1.2529297, 1.6640625, 1.6171875, 0.17529297, -0.01574707, 1.5771484, -0.48706055, 0.671875, 0.97021484, -0.6064453, -0.30322266, -0.01574707, -0.8261719, -1.6318359, 0.9633789, -1.7705078, 2.328125, -0.30151367, 0.6010742, -0.26391602, -0.91259766, -0.7241211, -0.5961914, -1.5673828, -0.27026367, 1.0361328, 0.3918457, 1.8125, 1.2060547, -0.097595215, 0.45043945, -0.28100586, 0.79248047, 0.56884766, -3.515625, -2.2871094, 0.11767578, -0.7241211, 0.16760254, -0.39038086, -0.004558563, 1.3164062, 0.15771484, -0.3527832, -0.8588867, -2.3691406, 0.0513916, 0.56689453, 2.328125, -0.16845703, 2.796875, 1.2695312, 0.5991211, 1.8720703, -3.4707031, -1.1650391, 2.5039062, -0.7011719, 0.6152344, 0.6479492, -1.0498047, -0.34692383, 1.734375, -0

In [63]:
len(sample_embedding)

4096

In [64]:
print(sample_embedding)
print(sample_embedding[0])

[0.6166992, 0.21386719, -0.022537231, -1.1689453, -1.0615234, 0.34960938, -1.8095703, -1.0517578, 0.47460938, 1.4521484, 0.7207031, 1.2783203, -1.8427734, -0.5415039, -1.15625, -0.8847656, 0.42773438, 1.4306641, 1.8125, -1.0712891, -1.2890625, -0.41918945, -1.2529297, 1.6640625, 1.6171875, 0.17529297, -0.01574707, 1.5771484, -0.48706055, 0.671875, 0.97021484, -0.6064453, -0.30322266, -0.01574707, -0.8261719, -1.6318359, 0.9633789, -1.7705078, 2.328125, -0.30151367, 0.6010742, -0.26391602, -0.91259766, -0.7241211, -0.5961914, -1.5673828, -0.27026367, 1.0361328, 0.3918457, 1.8125, 1.2060547, -0.097595215, 0.45043945, -0.28100586, 0.79248047, 0.56884766, -3.515625, -2.2871094, 0.11767578, -0.7241211, 0.16760254, -0.39038086, -0.004558563, 1.3164062, 0.15771484, -0.3527832, -0.8588867, -2.3691406, 0.0513916, 0.56689453, 2.328125, -0.16845703, 2.796875, 1.2695312, 0.5991211, 1.8720703, -3.4707031, -1.1650391, 2.5039062, -0.7011719, 0.6152344, 0.6479492, -1.0498047, -0.34692383, 1.734375, -0

# Step-4: Storing in Vector Stores

In [65]:
!pip install chromadb -q

In [66]:
# We can store the chunks generated convert them using embedding techniques inside a Chroma Database
from langchain.vectorstores import Chroma

In [67]:
patient_sample_db= Chroma.from_documents(chunks,
                                    embeddings, #Taking Cohere Embedding
                                    persist_directory="patient_sample_db" #create a dummy database
                          )
patient_sample_db.persist()

# Step-5: Retrieval

In [68]:
# Retrieve the documents that possibly answer the questions based on vector similarity
retriever = patient_sample_db.as_retriever()
result=retriever.get_relevant_documents("What are the diagonis for the patient"
                                      )
result

[Document(metadata={'page': 1, 'source': 'SampleMR.pdf'}, page_content='Please state your observations of the patient (i.e. the patient’s demeanour, body language, tone \nof voice etc. relevant to your assessment), the interactions you had with the patient/the tests'),
 Document(metadata={'page': 1, 'source': 'SampleMR.pdf'}, page_content='Please state your observations of the patient (i.e. the patient’s demeanour, body language, tone \nof voice etc. relevant to your assessment), the interactions you had with the patient/the tests'),
 Document(metadata={'page': 1, 'source': 'SampleMR.pdf'}, page_content='Please state your observations of the patient (i.e. the patient’s demeanour, body language, tone \nof voice etc. relevant to your assessment), the interactions you had with the patient/the tests'),
 Document(metadata={'page': 0, 'source': 'SampleMR.pdf'}, page_content='Full name of patient: Mr Tan Ah Kow \nNRIC/FIN/Passport no. of patient: S1111111X \nAge of patient: 55 years old \nSEC

In [69]:
# Fetching the metadata of the document fetched based on vector similarity
for i in range(len(result)):
  print(result[i].metadata)

{'page': 1, 'source': 'SampleMR.pdf'}
{'page': 1, 'source': 'SampleMR.pdf'}
{'page': 1, 'source': 'SampleMR.pdf'}
{'page': 0, 'source': 'SampleMR.pdf'}


You can also set a retrieval method that sets a similarity score threshold and only returns documents with a score above that threshold.


### pretty_print_docs

In [70]:
retriever = patient_sample_db.as_retriever()
result=retriever.get_relevant_documents("What is the name of the patient?",
                                        )
pretty_print_docs(result)

Document 1:

Patient’s clinical history: 
Please also state the source of the information (e.g. from medical records, from the patient, 
from the applicant etc.).
----------------------------------------------------------------------------------------------------
Document 2:

Patient’s clinical history: 
Please also state the source of the information (e.g. from medical records, from the patient, 
from the applicant etc.).
----------------------------------------------------------------------------------------------------
Document 3:

Patient’s clinical history: 
Please also state the source of the information (e.g. from medical records, from the patient, 
from the applicant etc.).
----------------------------------------------------------------------------------------------------
Document 4:

After being told he was in a hospital clinic, he identified me as a doctor, when asked. However, 
he could not remember my name, although I have been treating him for the last 5 years. He was


In [71]:
for i in range(len(result)):
  print(result[i].metadata)

{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 1, 'source': 'SampleMR.pdf'}


In [73]:
retriever = patient_sample_db.as_retriever()
result=retriever.get_relevant_documents("Age of the Patient")
result

[Document(metadata={'page': 0, 'source': 'SampleMR.pdf'}, page_content='Patient’s clinical history: \nPlease also state the source of the information (e.g. from medical records, from the patient, \nfrom the applicant etc.).'),
 Document(metadata={'page': 0, 'source': 'SampleMR.pdf'}, page_content='Patient’s clinical history: \nPlease also state the source of the information (e.g. from medical records, from the patient, \nfrom the applicant etc.).'),
 Document(metadata={'page': 0, 'source': 'SampleMR.pdf'}, page_content='Patient’s clinical history: \nPlease also state the source of the information (e.g. from medical records, from the patient, \nfrom the applicant etc.).'),
 Document(metadata={'page': 2, 'source': 'SampleMR.pdf'}, page_content='- 5 - \nDiagnosis: \n1. Dementia \n2. Stroke \nSECTION 4: OPINION ON PATIENT’S MENTAL CAPACITY \nOPINION ON PATIENT’S MENTAL CAPACITY IN RELATION TO PERSONAL WELFARE')]

In [74]:
for i in range(len(result)):
  print(result[i].metadata)

{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 0, 'source': 'SampleMR.pdf'}
{'page': 2, 'source': 'SampleMR.pdf'}


#### MultiQueryRetriever: Sometimes simple retriever can result in inaccurate result, but using multiquery retriever to generate variety of questions having same semantic can result in more accurate result.

In [79]:
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI , Cohere


llm=Cohere(temperature=0)

<ipython-input-79-77f61949807c>:6: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm=Cohere(temperature=0)


In [81]:
llm_based_retriver=MultiQueryRetriever.from_llm(
    retriever=patient_sample_db.as_retriever(),
    llm=llm
)

In [82]:
llm_based_retriver

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7a7248423160>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| Cohere(client=<cohere.client.Client object at 0x7a7254686b90>, async_client=<cohere.client.AsyncClient object at 0x7a725493caf0>, temperature=0.0, cohere_api_key=SecretStr('**********'))
| LineListOutputParser())

In [83]:
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [84]:
question1="What are the diagonis for the patient?"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)


INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different versions of the question : ', '1. What is the diagnosis of the patients past illnesses? ', '2. What are the possible diagnoses for the symptoms described by the patient? ', '3. Please provide me with a list of diagnostic possibilities for the patient, ranging from most to least likely based on the symptoms.']


In [85]:
rel_docs1

[Document(metadata={'page': 1, 'source': 'SampleMR.pdf'}, page_content='With regard to his orientation to time, place and person, he was unable to tell that he was in a \nhospital clinic but identified his son and was able to tell his own name and the name of his son.'),
 Document(metadata={'page': 1, 'source': 'SampleMR.pdf'}, page_content='you administered/the questions you posed, and the date you examined the patient. \nMr Tan was brought to the clinic in a wheel chair. His mood was euthymic (i.e. normal, non-'),
 Document(metadata={'page': 2, 'source': 'SampleMR.pdf'}, page_content='Date of physical examination/mental state examination: 20 June 2015 \nRelevant investigation results: \n[Results of CT brain, MRI brain scan and other investigations to be stated.] \n- 5 - \nDiagnosis:'),
 Document(metadata={'page': 0, 'source': 'SampleMR.pdf'}, page_content='Patient’s clinical history: \nPlease also state the source of the information (e.g. from medical records, from the patient, \nfro

#### Contextual compression to make the data used more efficient and precise for the model

In [86]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [87]:
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

compressor=LLMChainExtractor.from_llm(llm)

compression_retriever=ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=patient_sample_db.as_retriever()
)

In [88]:
compressed_docs=compression_retriever.get_relevant_documents(question1)

In [89]:
compressed_docs[0].metadata

{'page': 1, 'source': 'SampleMR.pdf'}

#### RetrievalQA Chain to validate the answers from RAG system

In [92]:
question1="What are the diagonis for the patient??"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Show me the possible diagnoses for this patient, ', ' 2. What illnesses or conditions might this patient have?, ', " 3. Help me identify potential diagnoses for this patient's symptoms."]


In [93]:
from langchain.chains import RetrievalQA
llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=llm_based_retriver
)

In [96]:
query="What is the name of patient?"
docs=Q_AChain({"query":query})
docs["result"]

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three alternative ways of asking this question that would still retrieve documents relevant to the original query: ', '1. What is the focus of the medical report involving the unnamed patient? ', "2. Please summarise the key details about the patient's identity. ", '3. What is the diagnosis of the patient?']


' The name of the patient is Mr. Tan. \n\nThis information can be found in the provided context in the line: "With regard to his orientation to time, place and person, he was unable to tell that he was in a hospital clinic but identified his son and was able to tell his own name and the name of his son." '

In [97]:
docs

{'query': 'What is the name of patient?',
 'result': ' The name of the patient is Mr. Tan. \n\nThis information can be found in the provided context in the line: "With regard to his orientation to time, place and person, he was unable to tell that he was in a hospital clinic but identified his son and was able to tell his own name and the name of his son." '}

In [98]:
print(Q_AChain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


#### Validation of the Results

In [117]:
#COI Data
!wget https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/sampleMR.pdf
from langchain.document_loaders import PyPDFLoader

#Document Loading
loader = PyPDFLoader("SampleMR.pdf")
pages = loader.load()

full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

#Split the data into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

#Embeddings and Vector DB
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma

embeddings = CohereEmbeddings(user_agent="langchain")
coi_db= Chroma.from_documents(chunks,
                             embeddings,
                             persist_directory="coi_db"
                             )
coi_db.persist()

#Retrieval Q_A Chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI , Cohere

llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever()
)

--2025-01-03 07:04:08--  https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/sampleMR.pdf
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-03 07:04:08 ERROR 404: Not Found.

Pages 4
Lines 151
Words 1690
Charecters 9429
8


In [119]:
query="""
Who is the doctor, and how long has the doctor been treating the patient
"""
docs=Q_AChain({"query":query})
print(docs["result"])

 The doctor is Dr. Tan Ah Moi. Tan has been treating the patient since November 2010. 


## RAG Validation Process

In [126]:
llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever(),
    input_key="question" # Add this extra parameter to access the user questions using input_key
)

##Get Validation data

In [127]:
!wget https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/Patient_Report_QA.csv

--2025-01-03 07:12:50--  https://github.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/raw/main/Patient_Report_QA.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/main/Patient_Report_QA.csv [following]
--2025-01-03 07:12:51--  https://raw.githubusercontent.com/harshitgupta1998/MedChain-Leveraging-AI-for-Medical-Applications/main/Patient_Report_QA.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1635 (1.6K) [text/plain]
Saving to: ‘Patient_Report_QA.csv.1’

Patient_Report_QA.c 100%[===================>]

In [128]:
import pandas as pd
test_data=pd.read_csv("Patient_Report_QA.csv")
test_data.head()

#Creating a iterative "Question", "Answer" pairs
test_qa_pairs = []
for index, row in test_data.iterrows():
  question = row['Question']
  answer = row['Answer']
  test_qa_pairs.append({'question': question, 'answer': answer})
print(test_qa_pairs)

[{'question': 'What is the full name and age of the patient?', 'answer': "The patient's name is Mr. Tan Ah Kow, and he is 55 years old."}, {'question': 'Who is the doctor, and how long has the doctor been treating the patient?', 'answer': 'The doctor is Dr. Tan Ah Moi, who has been treating the patient since November 2010.'}, {'question': "What is the patient's clinical history?", 'answer': 'Mr. Tan has a history of hypertension, hyperlipidemia, multiple strokes, cardiomyopathy, cardiac failure, and chronic renal disease. He also manifests behavioral and psychological symptoms secondary to dementia.'}, {'question': "What was the patient's demeanor during the examination?", 'answer': "Mr. Tan's mood was euthymic, and he did not exhibit psychotic symptoms."}, {'question': 'Was the patient oriented to time, place, and person?', 'answer': 'Mr. Tan was partially oriented. He recognized his son and himself but could not accurately determine the date, time, or place.'}, {'question': "How is t

In [135]:
#Add one more Question and Answer
question_new="what is black friday?"
answer_new="""
Black Friday is a shopping event that takes place on the Friday after Thanksgiving in the United States. It's a day when retailers offer deep discounts to kick off the holiday shopping season
"""
test_qa_pairs.append({'question': question_new, 'answer': answer_new})

## Get the Predictions

In [136]:
predictions = Q_AChain.apply(test_qa_pairs)
predictions

[{'question': 'What is the full name and age of the patient?',
  'answer': "The patient's name is Mr. Tan Ah Kow, and he is 55 years old.",
  'result': " The patient's full name is Mr. Tan Ah Kow and he is 55 years old. "},
 {'question': 'Who is the doctor, and how long has the doctor been treating the patient?',
  'answer': 'The doctor is Dr. Tan Ah Moi, who has been treating the patient since November 2010.',
  'result': ' The doctor is Dr. Tan Ah Moi. Tan has been treating the patient since November 2010. '},
 {'question': "What is the patient's clinical history?",
  'answer': 'Mr. Tan has a history of hypertension, hyperlipidemia, multiple strokes, cardiomyopathy, cardiac failure, and chronic renal disease. He also manifests behavioral and psychological symptoms secondary to dementia.',
  'result': ' The patient, Mr. Tan Ah Kow, has a history of hypertension, hyperlipidemia, multiple strokes, cardiomyopathy, cardiac failure, and chronic renal disease. He also manifests behavioral a

## Compare Actual Answers with predicted Answers

In [139]:
#The comparison is done using an LLM
from langchain.evaluation.qa import QAEvalChain

llm=Cohere(temperature=0)

qa_eval_chain = QAEvalChain.from_llm(llm)

In [140]:
eval_result = qa_eval_chain.evaluate(test_qa_pairs,
                                     predictions,
                                      question_key="question",
                                      answer_key="answer"
                                     )
print(eval_result)

[{'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' CORRECT'}, {'results': ' INCORRECT'}]


## Calculate The Accuracy

In [141]:
# Initialize counters
correct_count = 0
incorrect_count = 0

# Count occurrences
for item in eval_result:
    if item['results'].strip() == 'CORRECT':
        correct_count += 1
    elif item['results'].strip() == 'INCORRECT':
        incorrect_count += 1

# Calculate percentages
total_count = len(eval_result)
correct_percentage = (correct_count / total_count) * 100
incorrect_percentage = (incorrect_count / total_count) * 100

# Print the results
print(f"CORRECT: {correct_percentage}%")
print(f"INCORRECT: {incorrect_percentage}%")


CORRECT: 90.9090909090909%
INCORRECT: 9.090909090909092%
